<a href="https://colab.research.google.com/github/medinavi/2DTS/blob/main/RL_Exerc%C3%ADcio_1_Q_Learning_Cliff_Walking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Neste notebook, você codificará do zero seu segundo agente de Reinforcement Learning jogando Cliff Walking usando Q-Learning

Adaptado HuggingFace

<img src="https://www.gymlibrary.dev/_images/cliff_walking.gif" alt="Environments"/>

###🎮 Environments:

> 

- [CliffWalking-v0](https://www.gymlibrary.dev/environments/toy_text/cliff_walking/)


###📚 RL-Library: 

- Python and NumPy
- [Gym](https://www.gymlibrary.dev/)

## Instalar dependências e criar um display virtual 🔽


In [ ]:
!pip install gym==0.24
!pip install pygame
!pip install numpy

!pip install huggingface_hub
!pip install pickle5
!pip install pyyaml==6.0
!pip install imageio
!pip install imageio_ffmpeg
!pip install pyglet==1.5.1
!pip install tqdm

In [ ]:
%%capture
!sudo apt-get update
!apt install python-opengl ffmpeg xvfb
!pip3 install pyvirtualdisplay

Para garantir que as novas bibliotecas instaladas sejam usadas, **às vezes é necessário reiniciar o tempo de execução do notebook**. A próxima célula forçará o **tempo de execução a travar, então você precisará se conectar novamente e executar o código a partir daqui**.

In [ ]:
#import os
#os.kill(os.getpid(), 9)

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

## Importação de pacotes 📦

Além das bibliotecas instaladas, utilizamos também:

- `random`: Para gerar números aleatórios (que serão úteis para a política epsilon-greedy).
- `imageio`: Para gerar um vídeo de replay.

In [ ]:
import numpy as np
import gym
import random
import imageio
import os
import tqdm
import time

import pickle5 as pickle
from tqdm.notebook import tqdm

# Part 1: CliffWalking

## Criando o ambiente CliffWalking-v0 (https://www.gymlibrary.dev/environments/toy_text/cliff_walking/)
---

💡 Um bom hábito quando você começa a usar um ambiente é verificar sua documentação

👉 https://www.gymlibrary.dev/environments/toy_text/cliff_walking/

---


In [ ]:
#env = gym.make("CliffWalking-v0")

### Verifique o Environment:


In [ ]:
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space", env.observation_space)
print("Sample observation", env.observation_space.sample()) # Get a random observation

In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action

## Definindo os hiperparâmetros ⚙️
Os hiperparâmetros relacionados à exploração são alguns dos mais importantes.

- Precisamos garantir que nosso agente **explore o espaço de estados** o suficiente para aprender uma boa aproximação de valor. Para fazer isso, precisamos ter decaimento progressivo do epsilon.
- Se você diminuir o epsilon muito rápido (decay_rate muito alto), **você corre o risco de que seu agente fique preso**, já que seu agente não explorou o espaço de estado o suficiente e, portanto, não pode resolver o problema.

In [ ]:
state_space = env.observation_space.n
print("There are ", state_space, " possible states")

action_space = env.action_space.n
print("There are ", action_space, " possible actions")

In [ ]:
# Vamos criar nossa Qtable de tamanho (state_space, action_space) e inicializar cada valor em 0 usando np.zeros
def initialize_q_table(state_space, action_space):
  Qtable = np.zeros((state_space, action_space))
  return Qtable

In [ ]:
Qtable_CliffWalking = initialize_q_table(state_space, action_space)

In [ ]:
Qtable_CliffWalking

In [ ]:
def greedy_policy(Qtable, state):
  # Exploitation: take the action with the highest state, action value
  action = np.argmax(Qtable[state][:])
  
  return action

In [ ]:
def epsilon_greedy_policy(Qtable, state, epsilon):
  # Gera aleatoriamente um número entre 0 e 1
  random_int = random.uniform(0,1)
  # if random_int > maior que epsilon --> exploitation
  if random_int > epsilon:
     # Execute a ação com o maior valor dado um estado
     # np.argmax pode ser útil aqui
    action = greedy_policy(Qtable, state)
  # else --> exploration
  else:
    action = env.action_space.sample()
  
  return action

In [ ]:
# Parâmetros de treinamento
n_training_episodes = 10000 # Total de episódios de treinamento
learning_rate = 0.7 # Taxa de aprendizado

# Parâmetros de avaliação
n_eval_episodes = 100 # Número total de episódios de teste

# Parâmetros do ambiente
env_id = "CliffWalking-v0" # Nome do ambiente
max_steps = 99 # Max passos por episódio
gamma = 0.95 # Taxa de desconto
eval_seed = [] # A semente de avaliação do ambiente

# Parâmetros de exploração
max_epsilon = 1.0 # Probabilidade de exploração no início
min_epsilon = 0.05 # Probabilidade mínima de exploração
decay_rate = 0.0005 # Taxa de decaimento exponencial para prob de exploração

In [ ]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
  for episode in tqdm(range(n_training_episodes)):
    # # Reduzir epsilon (porque precisamos cada vez menos exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    # Redefinir o ambiente
    state = env.reset()
    step = 0
    done = False

    # repete
    for step in range(max_steps):
      # Escolha a ação At para usar a política gananciosa (greedy policy) do epsilon 
      action = epsilon_greedy_policy(Qtable, state, epsilon)

      # Take action At and observe Rt+1 and St+1
      # Take the action (a) and observe the outcome state(s') and reward (r)
      new_state, reward, done, info = env.step(action)

      # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
      Qtable[state][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])   

      # If done, finish the episode
      if done:
        break
      
      # Our next state is the new state
      state = new_state
  return Qtable

## Treinando o agente Q-Learning 🏃

In [ ]:
Qtable_CliffWalking = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable_CliffWalking)

## Verificando a tabela Q-Learning 👀

In [ ]:
Qtable_CliffWalking

## Avaliação do Método 📝

- Definimos o método de avaliação que vamos usar para testar nosso agente Q-Learning.

In [ ]:
def evaluate_agent(env, max_steps, n_eval_episodes, Q, seed):
  """
   Avalie o agente para episódios ``n_eval_episodes`` e retorne recompensa média e padrão de recompensa.
   :param env: O ambiente de avaliação
   :param n_eval_episodes: Número de episódios para avaliar o agente
   :param Q: A tabela Q
   :param seed: A matriz de sementes de avaliação (para taxi-v3)
   """
  episode_rewards = []
  for episode in tqdm(range(n_eval_episodes)):
    if seed:
      state = env.reset(seed=seed[episode])
    else:
      state = env.reset()
    step = 0
    done = False
    total_rewards_ep = 0
    
    for step in range(max_steps):
      # Tome a ação (índice) que tem a recompensa futura máxima esperada dado aquele estado
      action = greedy_policy(Q, state)
      new_state, reward, done, info = env.step(action)
      total_rewards_ep += reward
        
      if done:
        break
      state = new_state
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

## Avaliando nosso agente Q-Learning 📈


In [ ]:
# Evaluate our Agent
mean_reward, std_reward = evaluate_agent(env, max_steps, n_eval_episodes, Qtable_CliffWalking, eval_seed)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

#### Não modifique essa parte


In [ ]:
def play_actions(env, Qtable, delay = 1):
  """
   Gerar um vídeo de replay do agente
   :param env
   :param Qtable: Qtable do nosso agente
   :param out_directory
   :param fps: quantos quadros por segundo (com taxi-v3 e frozenlake-v1 usamos 1)
   """

  sequenses = []  
  done = False
  state = env.reset(seed=random.randint(0,500))
  txt = env.render(mode='human')

  sequenses.append(txt)
  while not done:
    # Tome a ação (índice) que tem a recompensa futura máxima esperada dado aquele estado
    
    action = np.argmax(Qtable[state][:])
    print(action)
    state, reward, done, info = env.step(action) # Colocamos diretamente next_state = state para a lógica de gravação
    txt = env.render(mode='human')
    sequenses.append(txt)
    print(txt)
    time.sleep(delay)
  
  return sequenses
 

In [ ]:
play_actions(env, Qtable_CliffWalking,  1)

In [ ]:
def record_video(env, Qtable, out_directory, fps=1):
  """
   Gerar um vídeo de replay do agente
   :param env
   :param Qtable: Qtable do nosso agente
   :param out_directory
   :param fps: quantos quadros por segundo (com taxi-v3 e frozenlake-v1 usamos 1)
   """
  images = []  
  done = False
  state = env.reset(seed=random.randint(0,500))
  img = env.render(mode='rgb_array')
  images.append(img)
  while not done:
    # Tome a ação (índice) que tem a recompensa futura máxima esperada dado aquele estado
    action = np.argmax(Qtable[state][:])
    state, reward, done, info = env.step(action) # Colocamos diretamente next_state = state para a lógica de gravação
    img = env.render(mode='rgb_array')
    images.append(img)
  imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)